<a href="https://colab.research.google.com/github/Haebuk/kuggle/blob/main/gamestop_before_sweep_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install wandb -qqq
import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split


In [57]:
PATH = '/content/drive/MyDrive/input/'
df = pd.read_csv(PATH + 'GME_scaled.csv')
print(df.shape)
df.head()

(2914, 3)


,date,open_price,volume
0,2010-01-04,3.138533,0.031204
1,2010-01-05,3.174297,0.024296
2,2010-01-06,3.197039,0.024552
3,2010-01-07,3.044999,0.206746
4,2010-01-08,3.074543,0.058121


In [58]:
default_config = {
    'learning_rate': 0.001,
    'dropout_rate': 0.2,
    'hidden1': 50,
    'hidden2': 50,
    'hidden3': 50,
    'time_step': 60,
}
wandb.init(project='gamestop_prediction', config=default_config)
config = wandb.config

epoch,14
loss,0.0042
mse,0.0042
val_loss,0.34595
val_mse,0.34595
_runtime,1099
_timestamp,1629354353
_step,14
best_val_loss,0.34595
best_epoch,14


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▃▃▂▂▂▂▁▁▂▁▁▁
val_mse,█▄▃▃▃▂▂▂▂▁▁▂▁▁▁
_runtime,▁▁▂▂▃▃▄▅▅▆▆▇▇██
_timestamp,▁▁▂▂▃▃▄▅▅▆▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█


In [59]:
X = df.drop('date', axis=1).values
y = df['open_price'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)
print(X_tr.shape, y_tr.shape)
print(X_te.shape, y_te.shape)

(2622, 2) (2622,)
(292, 2) (292,)


In [60]:
time_step = config.time_step
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(time_step,len(X_tr)): # train
    X_train.append(X_tr[i-time_step:i,:])
    y_train.append(y_tr[i])
for i in range(time_step, len(X_te)): # test
    X_test.append(X_te[i-time_step:i,:])
    y_test.append(y_te[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
# LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

In [62]:
model = tf.keras.Sequential([
    LSTM(config.hidden1, return_sequences=True, input_shape=(config.time_step, 2)),
    LSTM(config.hidden1, return_sequences=True),
    LSTM(config.hidden1),
    Dropout(config.dropout_rate),
    Dense(1),
])
opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(optimizer=opt,
              loss='mse',
              metrics=['mse'])
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=200, callbacks=[WandbCallback(), early_stopping])

Epoch 1/200
81/81 [==============================] - 7s 27ms/step - loss: 0.9850 - mse: 0.9850 - val_loss: 1.9742 - val_mse: 1.9742


wandb: ERROR Can't save model, h5py returned error: Layer ModuleWrapper has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/200
81/81 [==============================] - 1s 13ms/step - loss: 0.1252 - mse: 0.1252 - val_loss: 1.0504 - val_mse: 1.0504
Epoch 3/200
81/81 [==============================] - 1s 13ms/step - loss: 0.0892 - mse: 0.0892 - val_loss: 1.0921 - val_mse: 1.0921
Epoch 4/200
81/81 [==============================] - 1s 13ms/step - loss: 0.0818 - mse: 0.0818 - val_loss: 0.9509 - val_mse: 0.9509
Epoch 5/200
81/81 [==============================] - 1s 13ms/step - loss: 0.0703 - mse: 0.0703 - val_loss: 0.6837 - val_mse: 0.6837
Epoch 6/200
81/81 [==============================] - 1s 13ms/step - loss: 0.0743 - mse: 0.0743 - val_loss: 0.8101 - val_mse: 0.8101
Epoch 7/200
81/81 [==============================] - 1s 13ms/step - loss: 0.0684 - mse: 0.0684 - val_loss: 0.7787 - val_mse: 0.7787
Epoch 8/200
81/81 [==============================] - 1s 14ms/step - loss: 0.0637 - mse: 0.0637 - val_loss: 0.6744 - val_mse: 0.6744
Epoch 9/200
81/81 [==============================] - 1s 13ms/step - loss: 0.